In [18]:
import pandas as pd
from shapely import wkt,geometry
from shapely.geometry import Point
from shapely.geometry import LineString, Polygon
import networkx as nx
import numpy as np
import sumolib
import os

In [6]:
sumolib.os.environ["SUMO_HOME"]="/home/kaveh/build/sumo"
#sumolib.os.environ["SUMO_HOME"]="/usr/share/sumo"
if 'SUMO_HOME' in sumolib.os.environ:
    tools = sumolib.os.path.join(sumolib.os.environ['SUMO_HOME'], 'tools')
    sumolib.sys.path.append(tools)
else:   
    sumolib.sys.exit("please declare environment variable 'SUMO_HOME'")

sumoBinary = "/home/kaveh/notebook/jupyterenv/bin/sumo"

In [92]:
edgelist = pd.read_pickle("../new_networkx/graph_edgelist.pkl")
G = nx.from_edgelist(edgelist, create_using=nx.DiGraph)

In [93]:
net = sumolib.net.readNet('../map/osm.net.xml', withInternal=True)

In [94]:
id2type = nx.to_pandas_edgelist(G).set_index("id")["type"]

In [325]:
#route = pd.read_csv("../routes/routes.csv").drop(columns=["length","cost","speed"])

In [611]:
route = pd.read_csv("routes_1.csv")

In [612]:
route.head()

,vehicle_id,time,trip_id,from_node,to_node,path
0,0,0,22720,5354357258,21608373,618320304#0 -648762771#2 -648762771#1 -6487627...
1,1,0,4376,255364799,330043955,-225648631#5 -225648631#4 -225648631#2 -293825...
2,2,1,15851,330045345,976528927,-29981206#8 29981042 -29981047#3 -29981047#1 -...
3,3,2,694,1404484711,330041794,126777051#0 126777051#1 126777056#6 651768301#...
4,4,2,22242,514184883,330040554,-1033763162#1 -570986229#1 -839947528#1 -10337...


In [613]:
len(route)

10082

In [614]:
route = (route[route["from_node"]!=route["to_node"]])

In [615]:
#route = route.sort_values("time")

In [616]:
def route_df2xml(route, address):
    def convert_row(row):
        return """
        <vehicle id="%s" depart="%s" departLane="%s" departSpeed="%s">
            <route edges="%s"/>
        </vehicle>""" % (row.vehicle_id, row.time, "free", "max", row.path)

    text0 = """<?xml version="1.0" encoding="UTF-8"?>\n\n\n"""
    text1 = """<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">"""
    text2 = '\n'.join(route.apply(convert_row, axis=1))
    text3 = """\n</routes>"""
    with open(address, 'w') as myfile: 
        myfile.write(text0+text1+text2+text3)

In [617]:
#route_df2xml(route, "init_routes.xml")
route_df2xml(route, "routes_1.xml")

In [618]:
def create_edge_data_add(iteration):
        
    text0 = """<?xml version="1.0" encoding="UTF-8"?>\n\n\n"""
    text1 = """<additional xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/additional_file.xsd">
"""
    text2 = """
        <edgeData id="%s" freq="%s" file="%s" excludeEmpty="%s" withInternal="%s"/>"""%("dump_", 3600, "aggregated.xml", "true","true")

    text3 = """\n</additional>"""
    with open(str(iteration)+"/edge_data_add.xml", 'w') as myfile: 
        myfile.write(text0+text1+text2+text3)


In [624]:
def sumoCommand(iteration):
    diroutput = str(iteration)+"/"
    if (not sumolib.os.path.isdir(diroutput)):
        os.mkdir(diroutput,mode=0o777) 
    sc_dict = dict()
    sc_dict["--net-file"] = "../map/osm.net.xml"
    #sc_dict["--route-files"] = "init_routes.xml"
    sc_dict["--route-files"] = "routes_1.xml"

    sc_dict["--tripinfo-output"] = diroutput + "trip_info.xml"
    sc_dict["--statistic-output"] = diroutput + "statistic_output.xml"
    sc_dict["--vehroute-output"] =  diroutput  +"vehroute_output.xml"
    #sc_dict["--tripinfo-output.write-unfinished"] = "true"
    sc_dict["--tripinfo-output.write-undeparted"] = "true"
    sc_dict["--vehroute-output.write-unfinished"] = "true"
    #sc_dict["--vehroute-output.write-undeparted"] = "true"
    sc_dict["--additional-files"] = diroutput + "edge_data_add.xml"
    sc_dict["--vehroute-output.exit-times"] = "true"
    sc_dict["--no-warnings"] = "false"
    sc_dict["--no-step-log"] = "true"
    sc_dict["--end"] = "3600"
    sumoCmd = list()
    sumoCmd.append(sumoBinary)
    for key in sc_dict.keys():
        sumoCmd.append(key)
        sumoCmd.append(sc_dict[key])
    return sumoCmd


In [625]:
iteration = 1

In [626]:
scommand = sumoCommand(iteration)
scommand

['/home/kaveh/notebook/jupyterenv/bin/sumo',
 '--net-file',
 '../map/osm.net.xml',
 '--route-files',
 'routes_1.xml',
 '--tripinfo-output',
 '1/trip_info.xml',
 '--statistic-output',
 '1/statistic_output.xml',
 '--vehroute-output',
 '1/vehroute_output.xml',
 '--tripinfo-output.write-undeparted',
 'true',
 '--vehroute-output.write-unfinished',
 'true',
 '--additional-files',
 '1/edge_data_add.xml',
 '--vehroute-output.exit-times',
 'true',
 '--no-warnings',
 'false',
 '--no-step-log',
 'true',
 '--end',
 '3600']

In [622]:
create_edge_data_add(iteration)

In [623]:
sumolib.subprocess.call(scommand)

0

In [564]:
#sumolib.subprocess.call(["python3", tools+"/xml/xml2csv.py",
#                         str(iteration)+"/aggregated.xml",
#                            "--xsd",sumolib.os.environ["SUMO_HOME"] + "/data/xsd/meandata_file.xsd"])

sumolib.subprocess.call(["python3", tools+"/xml/xml2csv.py",
                         str(iteration)+"/aggregated.xml"])
aggregated = pd.read_csv(str(iteration)+"/aggregated.csv", sep=";")
   

In [565]:
aggregated.head()

,interval_begin,interval_end,interval_id,edge_arrived,edge_density,edge_departed,edge_entered,edge_id,edge_laneChangedFrom,edge_laneChangedTo,edge_laneDensity,edge_left,edge_occupancy,edge_overlapTraveltime,edge_sampledSeconds,edge_speed,edge_speedRelative,edge_timeLoss,edge_traveltime,edge_waitingTime
0,0.0,3600.0,dump_,0,0.40,0,11,:10981052_0,0,0,0.20,11,0.07,1.81,19.88,10.32,0.74,5.16,1.39,0.0
1,0.0,3600.0,dump_,0,1.58,0,56,:10981052_2,50,50,0.79,56,0.29,1.40,77.61,13.28,0.96,5.43,1.03,0.0
2,0.0,3600.0,dump_,0,0.35,0,11,:10981055_0,5,5,0.17,11,0.06,1.09,11.98,13.35,0.96,0.50,0.71,0.0
3,0.0,3600.0,dump_,0,1.72,0,41,:10981055_3,1,1,0.86,41,0.22,0.80,32.74,12.90,0.93,1.71,0.41,0.0
4,0.0,3600.0,dump_,0,0.35,0,11,:10981063_0,0,0,0.18,11,0.06,1.01,10.62,13.34,0.96,0.46,0.63,0.0


In [566]:
len(aggregated)

8039

In [567]:
alledges = set(aggregated["edge_id"])
aggregated = aggregated[aggregated["edge_traveltime"].notna()]

_edge2time = aggregated.set_index("edge_id")["edge_traveltime"].to_dict()

In [568]:
aggregated

,interval_begin,interval_end,interval_id,edge_arrived,edge_density,edge_departed,edge_entered,edge_id,edge_laneChangedFrom,edge_laneChangedTo,edge_laneDensity,edge_left,edge_occupancy,edge_overlapTraveltime,edge_sampledSeconds,edge_speed,edge_speedRelative,edge_timeLoss,edge_traveltime,edge_waitingTime
0,0.0,3600.0,dump_,0,0.40,0,11,:10981052_0,0,0,0.20,11,0.07,1.81,19.88,10.32,0.74,5.16,1.39,0.0
1,0.0,3600.0,dump_,0,1.58,0,56,:10981052_2,50,50,0.79,56,0.29,1.40,77.61,13.28,0.96,5.43,1.03,0.0
2,0.0,3600.0,dump_,0,0.35,0,11,:10981055_0,5,5,0.17,11,0.06,1.09,11.98,13.35,0.96,0.50,0.71,0.0
3,0.0,3600.0,dump_,0,1.72,0,41,:10981055_3,1,1,0.86,41,0.22,0.80,32.74,12.90,0.93,1.71,0.41,0.0
4,0.0,3600.0,dump_,0,0.35,0,11,:10981063_0,0,0,0.18,11,0.06,1.01,10.62,13.34,0.96,0.46,0.63,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8034,0.0,3600.0,dump_,0,24.98,0,1259,998248504#0,26,26,12.49,1258,5.43,2.46,3096.06,16.03,0.72,813.55,2.16,0.0
8035,0.0,3600.0,dump_,0,87.25,0,136,998323891#0,0,0,43.62,136,0.68,0.37,50.26,13.96,0.63,18.31,0.01,0.0
8036,0.0,3600.0,dump_,0,3.28,0,168,998323895,1,1,1.64,168,0.62,1.06,178.41,18.93,0.85,24.66,0.81,0.0
8037,0.0,3600.0,dump_,0,16.14,0,414,998333666#1,0,0,16.14,414,6.21,2.49,1031.08,9.13,0.41,601.17,2.04,0.0


In [569]:
G_df = nx.to_pandas_edgelist(G)

In [570]:
G_df

,source,target,id,speed,type,length,cost
0,from_-1001559563#1,to_-1001559563#1,-1001559563#1,2.78,real,9.441000e+01,3.396043e+01
1,to_-1001559563#1,from_-1001559570#1,:9244446810_0,2.78,internal,1.006000e+01,3.618705e+00
2,to_-1001559563#1,from_1001559570#2,:9244446810_1,2.78,internal,1.668000e+01,6.000000e+00
3,to_-1001559563#1,from_1001559563#0,:9244446810_2,2.78,internal,4.770000e+00,1.715827e+00
4,to_-1001559563#1,d_9244446810,to_-1001559563#1_d_9244446810,1.00,destination_node,1.000000e-07,1.000000e-07
...,...,...,...,...,...,...,...
38719,o_330043310,from_-829973541,o_330043310_from_-829973541,1.00,origin_node,1.000000e-07,1.000000e-07
38720,o_330043310,from_829973542#0,o_330043310_from_829973542#0,1.00,origin_node,1.000000e-07,1.000000e-07
38721,o_1242994659,from_-375367302#1,o_1242994659_from_-375367302#1,1.00,origin_node,1.000000e-07,1.000000e-07
38722,o_1242994659,from_-654504622,o_1242994659_from_-654504622,1.00,origin_node,1.000000e-07,1.000000e-07


In [571]:
G_ri = G_df[G_df["type"].apply(lambda x: x=="real" or x=="internal")]

In [572]:
def cost_update(G_df, net,iteration):
    sumolib.subprocess.call(["python3", tools+"/xml/xml2csv.py",str(iteration)+"/aggregated.xml"])
    aggregated = pd.read_csv(str(iteration)+"/aggregated.csv", sep=";")
    alledges = set(aggregated["edge_id"])
    aggregated = aggregated[aggregated["edge_traveltime"].notna()]

    _edge2time = aggregated.set_index("edge_id")["edge_traveltime"].to_dict()
    #for key in _edge2time.keys():
    #    edge = net.getEdge(key)
    #    if _edge2time[key] < edge.getLength()/edge.getSpeed():
    #        print(str(key) + "  :  "+str(_edge2time[key]) + "  " + str(edge.getLength()/edge.getSpeed()))
    
    for edge in net.getEdges():
        if not(edge.getID() in _edge2time.keys()):
            _edge2time[edge.getID()] = edge.getLength()/edge.getSpeed()
        else:
            _edge2time[edge.getID()] = max(_edge2time[edge.getID()], edge.getLength()/edge.getSpeed())
            
    _G_df = G_df.copy()
    _G_df["cost"] = G_df.apply(lambda row: _edge2time[row.id] if row.id in _edge2time.keys() else row.cost, axis=1)
    _G_df["speed"] = G_df.apply(lambda row: row.length/row.cost, axis =1)
    return _G_df

In [573]:
new_G_df = cost_update(G_df, net,iteration)

In [574]:
G_df

,source,target,id,speed,type,length,cost
0,from_-1001559563#1,to_-1001559563#1,-1001559563#1,2.78,real,9.441000e+01,3.396043e+01
1,to_-1001559563#1,from_-1001559570#1,:9244446810_0,2.78,internal,1.006000e+01,3.618705e+00
2,to_-1001559563#1,from_1001559570#2,:9244446810_1,2.78,internal,1.668000e+01,6.000000e+00
3,to_-1001559563#1,from_1001559563#0,:9244446810_2,2.78,internal,4.770000e+00,1.715827e+00
4,to_-1001559563#1,d_9244446810,to_-1001559563#1_d_9244446810,1.00,destination_node,1.000000e-07,1.000000e-07
...,...,...,...,...,...,...,...
38719,o_330043310,from_-829973541,o_330043310_from_-829973541,1.00,origin_node,1.000000e-07,1.000000e-07
38720,o_330043310,from_829973542#0,o_330043310_from_829973542#0,1.00,origin_node,1.000000e-07,1.000000e-07
38721,o_1242994659,from_-375367302#1,o_1242994659_from_-375367302#1,1.00,origin_node,1.000000e-07,1.000000e-07
38722,o_1242994659,from_-654504622,o_1242994659_from_-654504622,1.00,origin_node,1.000000e-07,1.000000e-07


In [575]:
new_G_df.describe()

,speed,length,cost
count,38724.000000,3.872400e+04,3.872400e+04
mean,5.861652,2.654061e+01,3.349582e+00
std,5.265239,8.976051e+01,1.312059e+01
min,0.006780,1.000000e-07,1.000000e-07
25%,1.000000,1.000000e-07,1.000000e-07
50%,3.650000,4.690000e+00,1.000000e+00
75%,8.610000,1.446000e+01,1.754924e+00
max,27.780000,5.314060e+03,6.411700e+02


In [576]:
G_df[G_df["id"]=="-225704060#3"]

,source,target,id,speed,type,length,cost
8531,from_-225704060#3,to_-225704060#3,-225704060#3,12.887466,real,472.97,36.7


In [577]:
new_G_df[new_G_df["id"]=="-225704060#3"]

,source,target,id,speed,type,length,cost
8531,from_-225704060#3,to_-225704060#3,-225704060#3,12.887466,real,472.97,35.57


In [578]:
len(route)

10082

In [579]:
len(route["from_node"].unique())

184

In [580]:
len(route["to_node"].unique())

184

In [581]:
##########################################################################

In [582]:
EPSILON = 0.0000001


def DAG_all2destination(G, destination_id):
    G_reverse = nx.reverse(G)
    ss1 = nx.single_source_dijkstra_path_length(G_reverse,source="d_"+str(destination_id), weight="cost")
    empty =np.inf
    nx.set_node_attributes(G_reverse, empty, "dist_from_destination")
    nx.set_node_attributes(G_reverse, ss1, "dist_from_destination")
  
    def filter_edge(n1, n2):
        if G_reverse.nodes[n1]["dist_from_destination"] ==np.inf:
            return False
        elif G_reverse.nodes[n2]["dist_from_destination"] ==np.inf:
            return False
        else:
            return G_reverse.nodes[n1]["dist_from_destination"] <= G_reverse.nodes[n2]["dist_from_destination"]

    def filter_node(n):
        return  G_reverse.nodes[n]["dist_from_destination"] !=np.inf

    view = nx.subgraph_view(G_reverse, filter_edge=filter_edge, filter_node=filter_node)
    DAG_reverse = view.copy()
    DAG = nx.reverse(DAG_reverse)
    return DAG

def filter_DAG_origin_based(DAG, origins):
    nodesorted = sorted(DAG.nodes, key=lambda n: DAG.nodes[n]['dist_from_destination'])
    nodesorted.reverse()
    poi = {"o_"+item for item in origins}
    nodedict = dict()
    for node in nodesorted:
        if (node in nodedict.keys()) or (node in poi):
            nodedict[node]=True
            neighbors_temp = list(nx.neighbors(DAG,node))
            for neighbor in neighbors_temp:
                nodedict[neighbor]=True
        else:
            nodedict[node]=False
            
    def filter_node_DAG(n):
        return  nodedict[n]

    view_DAG = nx.subgraph_view(DAG, filter_node=filter_node_DAG)
    DAG_filtered = view_DAG.copy()
    return DAG_filtered
 

In [583]:
def DAG_all2destination_od(G, destination_id, origins:set):
    if destination_id in origins:
        origins.remove(destination_id)
    DAG = DAG_all2destination(G, destination_id)
    DAG_filtered = filter_DAG_origin_based(DAG, origins)
    sorted_nodes = sorted(DAG_filtered.nodes, key=lambda n: DAG_filtered.nodes[n]['dist_from_destination'])
    return DAG_filtered, sorted_nodes

In [584]:
def create_vdict(DAG, sorted_nodes):
    neighbors = nx.to_dict_of_lists(DAG)
    neighbors_cost = dict()
    for node in neighbors.keys():
        tempdict = dict()
        for item in neighbors[node]:
            tempdict[item] = {"edgecost":DAG[node][item]["cost"]}
        neighbors_cost[node] = {"order":sorted_nodes.index(node),
                                "shortestpathcost":DAG.nodes[node]["dist_from_destination"],
                                "neighbors":tempdict}
    return neighbors_cost

In [585]:
def find_first_meet(vdict, nodesset):
    if len(nodesset)==0:
        print("Error: there is no node in the nodes set")
        return 0
    while len(nodesset)!=1:
        maxorder = -1
        maxnode = ""
        for node in nodesset:
            temp_order = vdict[node]["order"]
            if temp_order > maxorder:
                maxorder = temp_order
                maxnode = node
        nodesset.remove(maxnode)
        if "vout" not in vdict[maxnode].keys():
            print(maxnode)
            print(vdict[maxnode])
        else:
            nodesset.add(vdict[maxnode]["vout"])
    return nodesset.pop()

In [177]:
nx.to_pandas_edgelist(G)

,source,target,id,speed,type,length,cost
0,from_-1001559563#1,to_-1001559563#1,-1001559563#1,2.78,real,9.441000e+01,3.396043e+01
1,to_-1001559563#1,from_-1001559570#1,:9244446810_0,2.78,internal,1.006000e+01,3.618705e+00
2,to_-1001559563#1,from_1001559570#2,:9244446810_1,2.78,internal,1.668000e+01,6.000000e+00
3,to_-1001559563#1,from_1001559563#0,:9244446810_2,2.78,internal,4.770000e+00,1.715827e+00
4,to_-1001559563#1,d_9244446810,to_-1001559563#1_d_9244446810,1.00,destination_node,1.000000e-07,1.000000e-07
...,...,...,...,...,...,...,...
38719,o_330043310,from_-829973541,o_330043310_from_-829973541,1.00,origin_node,1.000000e-07,1.000000e-07
38720,o_330043310,from_829973542#0,o_330043310_from_829973542#0,1.00,origin_node,1.000000e-07,1.000000e-07
38721,o_1242994659,from_-375367302#1,o_1242994659_from_-375367302#1,1.00,origin_node,1.000000e-07,1.000000e-07
38722,o_1242994659,from_-654504622,o_1242994659_from_-654504622,1.00,origin_node,1.000000e-07,1.000000e-07


In [168]:
nx.to_pandas_edgelist(DAG_filtered)

,source,target
0,from_29420421#3,to_29420421#3
1,to_29420421#3,from_746404644
2,from_-726511409#4,to_-726511409#4
3,from_102555223,to_102555223
4,to_-726511409#4,from_-726511409#2
...,...,...
2769,o_330041794,from_29981225#0
2770,o_330042756,from_29980919#3
2771,o_2412315513,from_-35407770#4
2772,o_2546875470,from_975401579


In [586]:
def vcost_node2meet(vdict,node,meet):
    cost = 0
    nextnode = node
    while nextnode!=meet:

        cost += vdict[nextnode]["vcost"]
        nextnode = vdict[nextnode]["vout"]

    return cost

In [587]:
def calculate_vcost(vdict, node, teta):
    nodesset = set(vdict[node]["neighbors"].keys())

    if len(nodesset)==0:
        dict[node]["vcost"] = 0
    elif len(nodesset)==1:
        vdict[node]["vcost"] = EPSILON
        mynode = nodesset.pop()
        vdict[node]["vout"] = mynode
        vdict[node]["neighbors"][mynode]["prob"] = 1
    else:  
        meetnode = find_first_meet(vdict, nodesset)
        vdict[node]["vout"] = meetnode
        pi_od = vdict[node]["shortestpathcost"]-vdict[meetnode]["shortestpathcost"]
        expdict = dict()
        sumexp = 0
        for vertex in vdict[node]["neighbors"].keys():
            edgecost = vdict[node]["neighbors"][vertex]["edgecost"]


            cost = edgecost + vcost_node2meet(vdict,vertex,meetnode)# + vdict[meetnode]["shortestpathcost"]

            
            expdict[vertex] = np.exp(-teta*cost/pi_od)
            sumexp += expdict[vertex]

        for vertex in vdict[node]["neighbors"].keys():
            vdict[node]["neighbors"][vertex]["prob"] = expdict[vertex]/sumexp

        vdict[node]["vcost"] = np.abs(-(pi_od*sumexp)/teta)
    return vdict
    

In [627]:
def calculate_vcost_v2(vdict, node, teta, vdict_last):
    nodesset = set(vdict[node]["neighbors"].keys())

    if len(nodesset)==0:
        dict[node]["vcost"] = 0
    elif len(nodesset)==1:
        vdict[node]["vcost"] = EPSILON
        mynode = nodesset.pop()
        vdict[node]["vout"] = mynode
        vdict[node]["neighbors"][mynode]["prob"] = 1
    else:  
        meetnode = find_first_meet(vdict, nodesset)
        vdict[node]["vout"] = meetnode
        pi_od = vdict[node]["shortestpathcost"]-vdict[meetnode]["shortestpathcost"]
        expdict = dict()
        sumexp = 0
        for vertex in vdict[node]["neighbors"].keys():
            edgecost = vdict[node]["neighbors"][vertex]["edgecost"]


            cost = edgecost + vcost_node2meet(vdict,vertex,meetnode)# + vdict[meetnode]["shortestpathcost"]

            
            expdict[vertex] = np.exp(-teta*cost/pi_od)
            sumexp += expdict[vertex]
        w = dict()
        sumw = 0
        for vertex in vdict[node]["neighbors"].keys():
            w[vertex] = EPSILON
            if node in vdict_last.keys():
                if vertex in vdict_last[node]["neighbors"].keys():
                    w[vertex] = vdict[node]["neighbors"][vertex]["prob"]+EPSILON
                    
            vdict[node]["neighbors"][vertex]["prob"] = expdict[vertex]/sumexp
            w[vertex] = w[vertex]*vdict[node]["neighbors"][vertex]["prob"]
            sumw += w[vertex]
        for vertex in vdict[node]["neighbors"].keys():
            vdict[node]["neighbors"][vertex]["prob"] = w[vetex]/sumw
            
        vdict[node]["vcost"] = np.abs(-(pi_od*sumexp)/teta)
    return vdict


def calculate_prob_choice_v2(DAG, sorted_nodes, teta, vdict_last):
    vdict = create_vdict(DAG, sorted_nodes)
    destination = sorted_nodes[0]
    #sorted_nodes.remove(sorted_nodes[0])

    for node in sorted_nodes:
        if node!=destination:
            vdict = calculate_vcost_v2(vdict, node, teta, vdict_last)

    return vdict    

In [588]:
def calculate_prob_choice(DAG, sorted_nodes, teta):
    vdict = create_vdict(DAG, sorted_nodes)
    destination = sorted_nodes[0]
    #sorted_nodes.remove(sorted_nodes[0])

    for node in sorted_nodes:
        if node!=destination:
            vdict = calculate_vcost(vdict, node, teta)

    return vdict    

In [589]:
G = nx.from_pandas_edgelist(new_G_df, edge_attr=True, create_using=nx.DiGraph)
#od = pd.read_csv("../od/od.csv")
#od = od[od["trips"]>0].reset_index(drop=True)
destination_id = "330046001"
origins = set(route[route["to_node"]==destination_id]["from_node"])
DAG_filtered, sorted_nodes = DAG_all2destination_od(G, destination_id, origins)


In [590]:
id2type = new_G_df.set_index("id")["type"]


In [591]:
dtree = calculate_prob_choice(DAG_filtered, sorted_nodes, teta=2.5)

In [593]:
def nodepath2edgepath(path):
    path_ = path.copy()
    edgelist = list()
    x = path_[0]
    path_.remove(x)
    for node in path_:
        edgelist.append(G[x][node]["id"])
        x = node
    return " ".join(edgelist)

def fixpath(path, id2type):
    path = path.split(" ")
    for item in path:
        if id2type[item]!="real":
            path.remove(item)
    path = " ".join(path)  
    return path

def decisiontree2path_origin(dtree, origin):
    path = list()
    origin = "o_"+ str(origin)
    path.append(origin)
    nextnodesdict = dtree[origin]["neighbors"]
    while len(nextnodesdict)!=0:
        if len(nextnodesdict)==1:
            nextnode = list(nextnodesdict)[0]
        else:
            nextnode = random.choices(list(nextnodesdict),
                                      [ nextnodesdict[item]["prob"] for item in nextnodesdict.keys()])[0]
        path.append(nextnode)
        nextnodesdict = dtree[nextnode]["neighbors"]
    edgepath = nodepath2edgepath(path)
    return fixpath(edgepath, id2type)

def decisiontree2path_routedf(dtree, destination, routedf):
    myroutes = routedf[routedf["to_node"]==destination].copy().reset_index(drop=True)
    for index,row in myroutes.iterrows():
        newpath = decisiontree2path_origin(dtree, row.from_node)
        myroutes.loc[index,"path"] = newpath
    return myroutes

In [594]:
decisiontree2path_routedf(dtree, destination_id, route)

,vehicle_id,time,trip_id,from_node,to_node,path
0,320,109,8130,330036595,330046001,29981201#0 29981201#3 -29981143#3 -29981143#2 ...
1,532,185,14905,330044936,330046001,29981296#23 29981296#24 29981360#4 29981360#5 ...
2,612,213,28338,cluster_330041145_330044386,330046001,-548069720#0 733057875#3 733057875#1 -99794910...
3,661,227,10056,330041440,330046001,104797980#0 29981383#6 -29981307#5 -996548637#...
4,733,259,12224,330043023,330046001,-29980989#0 29980936#8 -29981159#1 -29981148#6...
...,...,...,...,...,...,...
59,9820,3504,6402,3037609402,330046001,750716319#0 975415107 918401796#0 918401796#1 ...
60,9916,3540,11318,330042642,330046001,-654504611 -34614075#10 -34614075#8 -34614075#...
61,9957,3555,4860,2670815505,330046001,261463330#7 -82164708#5 -82164708#3 -82164708#...
62,10049,3586,24683,6304257800,330046001,-223138918#1 223138920#0 975401569 -997528349#...


In [595]:
destinations = set(route["to_node"].unique())

df_list = list()
for destination_id in destinations:
    origins = set(route[route["to_node"]==destination_id]["from_node"])
    DAG_filtered, sorted_nodes = DAG_all2destination_od(G, destination_id, origins)
    dtree = calculate_prob_choice(DAG_filtered, sorted_nodes, teta=2.5)
    df_list.append(decisiontree2path_routedf(dtree, destination_id, route))
newroute = pd.concat(df_list)

329983499
330045423
2316163380
8132449074
3878025247
332236883
330046001
2559588578
6113530410
330040956
4452226084
1199475134
514184920
330045927
6161985947
330041084
6329296611
330045486
330042756
5490669837
330042713
330043393
4027663361
330043171
330045400
361191300
6793225631
6304257804
2670815505
330046003
323378327
6770075859
330042596
514184883
330043310
2546875470
330040701
5022920587
330042413
3732913490
6126996586
330043955
6989036711
976528927
330043625
330045111
360477824
cluster_330041132_330267660_418091425
6138125018
2344573460
cluster_291381188_330041233
414271220
360474078
3681387999
360438835
2167263401
330044636
330040554
6778082404
8220941
330036595
6111414502
323250291
4871254737
3139882142
6161985980
332639850
2670815564
330041106
21608373
297305382
323378174
1157945201
4813181314
1404484723
330043064
6304257800
2412315513
cluster_330045496_330045504_5200254966_5200254968
4812389444
360439481
3787401132
330044693
330043266
330045454
cluster_330045282_330045762
33

In [ ]:
def parallel_sampling(report, od_estimation, routes, option, interval, iteration, sample_iteration, dod,
                      last_best_iteration, last_best_sample_iteration, net, oldedge2time,return_dict):

    report["sample_number"] = sample_iteration
    vehid2pathid, all_routes, report["vehicle_number_sampling"] = route_sampling(od_estimation, routes, option, interval, iteration, sample_iteration)  
    create_edge_data_add(option, interval, iteration, sample_iteration)
    simulation(option, interval, iteration, dod, last_best_iteration, last_best_sample_iteration, net, sample_iteration)
    newedge2time = fun_edge2time(net, option, interval, iteration, sample_iteration)
    error_sensor, report["sensor_error_calibration"], report["sensor_error_sim2calib"] = \
                                         fun_sensor_error(option, interval, iteration, sample_iteration)
    report["sensor_error_simulation"] = error_sensor
    viz_edge_count(routes, od_estimation, net, option, interval, iteration, sample_iteration, geo=False)
    running, waiting, average_speed, teleport = logging_statistics(option, interval, iteration, sample_iteration)
    report["running"] = running
    report["waiting"] = waiting
    report["speed"] = average_speed
    report["fixed_point_error"] = fixedpoint_error(oldedge2time, newedge2time)
    report["teleport"] = teleport
    report["newedge2time"] = newedge2time
    #report["newroutes"] = new_routes
    report["all_routes"] = all_routes
    return_dict[sample_iteration] = report 
    
###############################################################################################    

manager = multiprocessing.Manager()
return_dict = manager.dict()
oldedge2time = edge2time_list[-1]
processes = []

for sample_iteration in range(int(option.sample_iteration)):
        p = multiprocessing.Process(target = parallel_sampling, args=(report.copy(), od_estimation, routes, option, interval, iteration,
                                                                      sample_iteration, dod,last_best_iteration,
                                                                      last_best_sample_iteration, net, oldedge2time, return_dict))
        p.start()
        processes.append(p)

for p in processes:
        p.join()    



best_sample = -1
best_sample_sensor_error = 1000

for i in range(len(return_dict)):
    if return_dict[i]["sensor_error_simulation"] < best_sample_sensor_error:
        best_sample_sensor_error = return_dict[i]["sensor_error_simulation"]
        best_sample = i

                
                
with open(option.output_location+"report.csv", 'a', encoding='UTF8', newline='') as f:
                        writer = csv.DictWriter(f, fieldnames=report_fieldnames)
                        for i in range(len(return_dict)):
                            myreport = dict()
                            for key in report_fieldnames:
                                myreport[key] = return_dict[i][key]
                            writer.writerow(myreport)  

In [608]:
newroute

,vehicle_id,time,trip_id,from_node,to_node,path
3728,0,0,22720,5354357258,21608373,618320304#0 -648762771#2 -648762771#1 -6487627...
2186,1,0,4376,255364799,330043955,-225648631#5 -225648631#4 -225648631#2 -293825...
2246,2,1,15851,330045345,976528927,-29981206#8 29981042 -29981047#3 -29981047#1 -...
6729,3,2,694,1404484711,330041794,126777051#0 126777051#1 126777056#6 651768301#...
3028,4,2,22242,514184883,330040554,-1033763162#1 -570986229#1 -839947528#1 -10337...
...,...,...,...,...,...,...
3905,10079,3597,23372,578673912,297305382,-1033630305#5 -1033630305#2 142955641#2 568629...
2440,10080,3597,12946,330043310,330045111,-829973541 -223747998#8 -223747998#5 -22374799...
9709,10081,3598,3123,2412315513,578673912,-35407770#4 -35407770#3 -35407770#1 -35407770#...
6206,10082,3598,1427,2110802894,1404484711,-1033622463#1 -1033622463#0 -1033622464#5 -103...


In [609]:
newroute.to_csv("routes_1.csv",index=False)